# Instructions
The following code was designed to be used in conjunction with FreezeAnalysis.ipynb and FreezeAnalysis_BatchProcess.ipynb in order to set the motion threshold for detecting motion.  Videos should be loaded in which an animal is not present in the box.


### Package Requirements
The following will need to be installed in your Conda environment:

- python (3.6.5)

- jupyter

- imread

- mahotas(1.4.4)

- numpy(1.14.3)

- pandas(0.23.0)

- matplotlib(2.2.2). 

- opencv(3.4.3

The following commands can be executed in your terminal to create an environment with these packages: 

- conda config --add channels conda-forge

- conda create -n EnvironmentName python=3.6.5 mahotas=1.4.4 pandas=0.23.0 matplotlib=2.2.2 opencv=3.4.3 jupyter imread


# 1. Load Necessary Packages

In [ ]:
import pylab 
import os
import cv2
import numpy as np
import mahotas as mh 
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

# 2. Set Directory and File Information
### To be set by user:

In [ ]:
#Video information
dpath = "/Users/ZP/Videos" # directory containing file
file = "Calibration Box 1.mpg" #filename. will take mpg and wmv files but maybe more.  only mpg1 have been extensively tested.
cal_sec = 8 #number of seconds in video to calibrate based upon
fps = 30 #frames per second

#Calibration parameters.  Needn't be changed.
SIGMA = 1 #this is sigma used for gaussian smoothing of image.  Used to reduce influence of frame by frame pixel jitter. 1 works well but could increased slightly if noise is detected. 
cal_pix = 10000 #set number of pixels to be examined.  10,000 works well.
Rmv_Animal = False #True/False. Attempt to remove influence of animal.  Only set to True if animal is in box

# 3. Load Video Information.  Display First Frame

In [ ]:
#Upoad file
fpath = dpath + "/" + file
print('file: '+ fpath)
cap = cv2.VideoCapture(fpath)

#Get maxiumum frame of file. Note that this is updated later if fewer frames detected
cap_max = int(cap.get(7)) #7 is index of total frames
print('total frames: ' + str(cap_max))

#Set first frame to be displayed
cap.set(1,0) #first index references frame property, second specifies next frame to grab

#Load first frame
ret, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#Present frame
plt.gray()
plt.figure(figsize=(8,8))
plt.title('first frame: ' + str(0))
plt.imshow(gray)

# 4. Calibrate Video 
###### The following code will select cal_pix pixels (10,000, by default) at random and look at how their pixel values change across the specified length of the video.  By looking at the distribution of frame-by-frame change values for each pixel, a threshold is then set for determining what changes are likely to be attributable to the animal moving versus random fluctuation.  Currently, cutoff is is set to twice the 99.99 percentile.  

This should be performed on a video with no animal inside the box, with Rmv_Animal set to False.  If multiple contexts are to be used, it is best to eventually use the same motion threshold across contexts, either taking the average of the various contexts or taking the maxiumum.  

Alternatively, there is currently an option to take video with the animal in the box and try to reduce the influence of its movement, but it cannot do so entirely, and thresholds are generally much higher.  You will likely need to play with how motion threshold is set if you do this. If animal is in the box, Rmv_Animal must be set to True.  Again, calbrating with the animal in the box is not recommended.

In [ ]:
#set seconds to examine and frames
cal_frames = cal_sec*fps

#Initialize matrix for difference values
cal_dif = np.zeros((cal_frames,cal_pix))

#Re-initialize video
cap.set(1,0) #first index references frame property, second specifies next frame to grab

#Initialize first frame
ret, frame = cap.read()
frame_new = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
frame_new = mh.gaussian_filter(frame_new,sigma=SIGMA)

#Get random set of pixels to examine across frames
h,w=frame_new.shape
h_loc = np.random.rand(cal_pix,1)*h
h_loc = h_loc.astype(int)
w_loc = np.random.rand(cal_pix,1)*w
w_loc = w_loc.astype(int)

#Loop through frames to detect frame by frame differences
for x in range (1,cal_frames):

    #Reset old frame
    frame_old = frame_new
    
    #Load next frame
    ret, frame = cap.read()

    #Process frame
    frame_new = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_new = mh.gaussian_filter(frame_new,sigma=SIGMA) # used to reduce influence of jitter from one frame to the next
    
    #Get differences for select pixels
    frame_pix_dif = np.absolute(frame_new[h_loc,w_loc] - frame_old[h_loc,w_loc])
    frame_pix_dif = frame_pix_dif[:,0]
     
    #Populate difference array
    cal_dif[x,:]=frame_pix_dif

    
#Attempt to remove differences due to movement of animal
if Rmv_Animal:
    otsu_rmv_animal = (mh.otsu((cal_dif*1000).astype('uint64')))/1000 #otsu will not work on non-integers.  This allows rounding to nearest 1000
    print('Otsu to remove animal: ' + str(otsu_rmv_animal))
    ninetynine_point_ninenine = np.percentile(cal_dif[cal_dif<otsu_rmv_animal],99.99)
else:
    ninetynine_point_ninenine = np.percentile(cal_dif,99.99)      
    
#Calculate grayscale change cutoff for detecting motion
cal_dif_avg = np.nanmean(cal_dif)

#Set Cutoff
mt_cutoff = 2*ninetynine_point_ninenine

#Print stats and selected cutoff
print ('Average frame-by-frame pixel difference: ' + str(cal_dif_avg))
print ('99.99 percentile of pixel change differences: ' + str(ninetynine_point_ninenine))
print ('Grayscale change cut-off for pixel change: ' + str(mt_cutoff))
